In [146]:
import pandas as pd
import numpy as np
import hvplot.pandas
from sklearn.cluster import KMeans
import plotly.express as px

In [147]:
file_path_data = "Resources/Data/2021VAERSDATA.csv"
encoding = 'unicode_escape'
VAERS_DATA_df = pd.read_csv(file_path_data, encoding='unicode_escape')
VAERS_DATA_df.head()

,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,DIED,DATEDIED,L_THREAT,...,CUR_ILL,HISTORY,PRIOR_VAX,SPLTTYPE,FORM_VERS,TODAYS_DATE,BIRTH_DEFECT,OFC_VISIT,ER_ED_VISIT,ALLERGIES
0,916710,1/1/2021,MO,23.0,23.0,NaN,F,NaN,NaN,Y,...,NaN,Hypothyroidism,NaN,NaN,2,1/1/2021,NaN,NaN,Y,NKDA
1,916720,1/1/2021,NY,23.0,23.0,NaN,M,NaN,NaN,NaN,...,NaN,NaN,unknown,NaN,2,1/1/2021,NaN,NaN,Y,NaN
2,916741,1/1/2021,AR,68.0,68.0,NaN,F,NaN,NaN,NaN,...,had surgery R hand for advanced arthritis 11/1...,Rheumatoid arthritis - mostly affecting R wris...,NaN,NaN,2,1/1/2021,NaN,NaN,NaN,bee stings
3,916772,1/1/2021,GA,55.0,55.0,NaN,M,NaN,NaN,NaN,...,NaN,"HTN, Insomnia,High Cholesterol,",NaN,NaN,2,1/1/2021,NaN,Y,NaN,Codeine
4,916790,1/1/2021,TN,52.0,52.0,NaN,F,NaN,NaN,Y,...,NaN,"Asthma, migraines",NaN,NaN,2,1/1/2021,NaN,NaN,Y,"Sulfa, shellfish"


In [148]:
file_path_vax = "Resources/Data/2021VAERSVAX.csv"
encoding = 'unicode_escape'
VAERS_VAX_df = pd.read_csv(file_path_vax, encoding= 'unicode_escape')
VAERS_VAX_df.head()

,VAERS_ID,VAX_TYPE,VAX_MANU,VAX_LOT,VAX_DOSE_SERIES,VAX_ROUTE,VAX_SITE,VAX_NAME
0,916710,COVID19,MODERNA,NaN,1,IM,LA,COVID19 (COVID19 (MODERNA))
1,916720,COVID19,MODERNA,011L20A,1,IM,AR,COVID19 (COVID19 (MODERNA))
2,916741,COVID19,PFIZER\BIONTECH,EH9899,1,SYR,LA,COVID19 (COVID19 (PFIZER-BIONTECH))
3,916772,COVID19,PFIZER\BIONTECH,EJ1685,UNK,IM,LA,COVID19 (COVID19 (PFIZER-BIONTECH))
4,916790,COVID19,PFIZER\BIONTECH,NaN,1,IM,RA,COVID19 (COVID19 (PFIZER-BIONTECH))


In [149]:
#Drop Unnecessary columns from VAERS_VAX_df
VAERS_VAX_df= VAERS_VAX_df.drop(["VAX_TYPE", "VAX_LOT", "VAX_ROUTE", "VAX_SITE", "VAX_NAME"], axis=1).copy()
VAERS_VAX_df.head(20)

,VAERS_ID,VAX_MANU,VAX_DOSE_SERIES
0,916710,MODERNA,1
1,916720,MODERNA,1
2,916741,PFIZER\BIONTECH,1
3,916772,PFIZER\BIONTECH,UNK
4,916790,PFIZER\BIONTECH,1
5,916803,SANOFI PASTEUR,4
6,916809,PFIZER\BIONTECH,1
7,916836,MODERNA,1
8,916859,MODERNA,1
9,916890,PFIZER\BIONTECH,1


In [150]:
file_path_symptoms = "Resources/Data/2021VAERSSYMPTOMS.csv"
encoding = 'unicode_escape'
VAERS_SYMPTOMS_df = pd.read_csv(file_path_symptoms, encoding= 'unicode_escape')
VAERS_SYMPTOMS_df.head()

,VAERS_ID,SYMPTOM1,SYMPTOMVERSION1,SYMPTOM2,SYMPTOMVERSION2,SYMPTOM3,SYMPTOMVERSION3,SYMPTOM4,SYMPTOMVERSION4,SYMPTOM5,SYMPTOMVERSION5
0,916710,Appendicitis,23.1,Band neutrophil percentage increased,23.1,Surgery,23.1,White blood cell count increased,23.1,NaN,NaN
1,916720,Dysphagia,23.1,Dyspnoea,23.1,NaN,NaN,NaN,NaN,NaN,NaN
2,916741,Chills,23.1,Complex regional pain syndrome,23.1,Fatigue,23.1,Headache,23.1,Joint range of motion decreased,23.1
3,916741,Myalgia,23.1,Pain in extremity,23.1,Peripheral swelling,23.1,X-ray abnormal,23.1,NaN,NaN
4,916772,Feeling abnormal,23.1,SARS-CoV-2 test positive,23.1,NaN,NaN,NaN,NaN,NaN,NaN


In [151]:
VAERS_DATA_df.columns

Index(['VAERS_ID', 'RECVDATE', 'STATE', 'AGE_YRS', 'CAGE_YR', 'CAGE_MO', 'SEX',
       'DIED', 'DATEDIED', 'L_THREAT', 'ER_VISIT', 'HOSPITAL', 'HOSPDAYS',
       'X_STAY', 'DISABLE', 'RECOVD', 'VAX_DATE', 'ONSET_DATE', 'NUMDAYS',
       'LAB_DATA', 'V_ADMINBY', 'V_FUNDBY', 'OTHER_MEDS', 'CUR_ILL', 'HISTORY',
       'PRIOR_VAX', 'SPLTTYPE', 'FORM_VERS', 'TODAYS_DATE', 'BIRTH_DEFECT',
       'OFC_VISIT', 'ER_ED_VISIT', 'ALLERGIES'],
      dtype='object')

In [153]:
VAERS_DATA_df=VAERS_DATA_df.drop(['RECVDATE', 'STATE', 'CAGE_YR', 'CAGE_MO', 'LAB_DATA', 'DATEDIED', 'L_THREAT', 'ER_VISIT', 'HOSPDAYS', 'X_STAY', 'VAX_DATE', 'V_ADMINBY', 'V_FUNDBY', 'ER_ED_VISIT', 'OFC_VISIT', 'FORM_VERS', 'BIRTH_DEFECT', 'DISABLE', 'RECOVD', 'PRIOR_VAX', 'ONSET_DATE', 'NUMDAYS', 'SPLTTYPE', 'TODAYS_DATE'], axis=1).copy()
VAERS_DATA_df.head(50)

,VAERS_ID,AGE_YRS,SEX,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES
0,916710,23.00,F,NaN,Y,Synthroid,NaN,Hypothyroidism,NKDA
1,916720,23.00,M,NaN,NaN,NaN,NaN,NaN,NaN
2,916741,68.00,F,NaN,NaN,phenobarbital 60mg HS hydroxychloroquin 400mg...,had surgery R hand for advanced arthritis 11/1...,Rheumatoid arthritis - mostly affecting R wris...,bee stings
3,916772,55.00,M,NaN,NaN,"Propranolol,zoloft,asa,Lisinipril, Crestor, Pr...",NaN,"HTN, Insomnia,High Cholesterol,",Codeine
4,916790,52.00,F,NaN,NaN,"Estradiol, topiramate, Emgality, sumatriptan, ...",NaN,"Asthma, migraines","Sulfa, shellfish"
5,916803,78.00,M,Y,Y,"Lisinopril, Novolog,, Lantus Solostar, Gabapen...",none,"large T-cell lymphoma, HTN, Gout, recieving t...",N?A
6,916809,40.00,F,NaN,Y,NaN,None,NaN,None
7,916836,55.00,M,NaN,Y,No current medications,Pt reported frequent urination for several months,None,No known allergies
8,916859,37.00,F,NaN,NaN,"fexofenadine, famotidine, hydroxychloroquine,",None,"Undifferentiated connective tissue disorder, a...","peanuts, soy, oysters, crab/lobster, penicillin"
9,916890,39.00,F,NaN,Y,"Zoloft, vitamin d, vitamin b, zinc, Wellbutrin...",None,None until vaccine,None


In [154]:
VAERS_DATA_df.columns

Index(['VAERS_ID', 'AGE_YRS', 'SEX', 'DIED', 'HOSPITAL', 'OTHER_MEDS',
       'CUR_ILL', 'HISTORY', 'ALLERGIES'],
      dtype='object')

In [155]:
VAERS_DATA_df.dtypes

VAERS_ID        int64
AGE_YRS       float64
SEX            object
DIED           object
HOSPITAL       object
OTHER_MEDS     object
CUR_ILL        object
HISTORY        object
ALLERGIES      object
dtype: object

In [156]:
#Transform String Columns
def change_string(died):
    if died == "Y":
        return 1
    else:
        return 0 

VAERS_DATA_df["DIED"]= VAERS_DATA_df["DIED"].apply(change_string)
VAERS_DATA_df.head(20)

,VAERS_ID,AGE_YRS,SEX,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES
0,916710,23.00,F,0,Y,Synthroid,NaN,Hypothyroidism,NKDA
1,916720,23.00,M,0,NaN,NaN,NaN,NaN,NaN
2,916741,68.00,F,0,NaN,phenobarbital 60mg HS hydroxychloroquin 400mg...,had surgery R hand for advanced arthritis 11/1...,Rheumatoid arthritis - mostly affecting R wris...,bee stings
3,916772,55.00,M,0,NaN,"Propranolol,zoloft,asa,Lisinipril, Crestor, Pr...",NaN,"HTN, Insomnia,High Cholesterol,",Codeine
4,916790,52.00,F,0,NaN,"Estradiol, topiramate, Emgality, sumatriptan, ...",NaN,"Asthma, migraines","Sulfa, shellfish"
5,916803,78.00,M,1,Y,"Lisinopril, Novolog,, Lantus Solostar, Gabapen...",none,"large T-cell lymphoma, HTN, Gout, recieving t...",N?A
6,916809,40.00,F,0,Y,NaN,None,NaN,None
7,916836,55.00,M,0,Y,No current medications,Pt reported frequent urination for several months,None,No known allergies
8,916859,37.00,F,0,NaN,"fexofenadine, famotidine, hydroxychloroquine,",None,"Undifferentiated connective tissue disorder, a...","peanuts, soy, oysters, crab/lobster, penicillin"
9,916890,39.00,F,0,Y,"Zoloft, vitamin d, vitamin b, zinc, Wellbutrin...",None,None until vaccine,None


In [109]:
###############

In [110]:
##Initializing model with K=3
model = KMeans(n_clusters=3, random_state=5)
model

KMeans(n_clusters=3, random_state=5)

In [ ]:
# Fitting model
model.fit(VAERS_DATA_df)

In [ ]:
# Get the predictions
predictions = model.predict(VAERS_DATA_df)
print(predictions)

In [ ]:
# Add a new class column to the VAERS_DATA_df
VAERS_DATA_df["class"] = model.labels_
VAERS_DATA_df.head()

In [ ]:
# Creating a scatterplot of VAERS_DATA_df
#Using features 
df_iris.hvplot.scatter(x="SEX", y="HOSPITAL", by="class")

In [ ]:
#Plotting the clusters with three features
fig = px.scatter_3d(
VAERS_DATA_df,
x="SEX",
y="HOSPITAL",
z="CUR_ILL",
color="class",
symbol="class",
size="sepal_width",
width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [ ]:
#Function to cluster and plot dataset
def test_cluster_amount(df, clusters):
    model = KMeans(n_clusters=clusters, random_state=5)
    model

    # Fitting model
    model.fit(df)

    # Add a new class column to VAERS_DATA_DF
    df["class"] = model.labels_

In [ ]:
##Test Cluster Amounts
test_cluster_amount(VAERS_DATA_df, 2)
VAERS_DATA_df.hvplot.scatter(x="SEX", y="HOSPITAL", by="class")

In [ ]:
## Use Elbow Curve to find best number of clusters
# Create empty list to hold inertia values
# Store a range of K values to test

inertia = []
k = list(range(1,11))

In [ ]:
# Looking for the best K
for i in k:
km = KMeans(n_clusters=i, random_state=0)
    km.fit(VAERS_DATA_df)
inertia.append(km.inertia_)

In [ ]:
# Define a DataFrame to plot the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

In [ ]:
##Look at the graph generated and determine correct amount of clusters

In [ ]:
# Create a K-means function to reuse the K-means cluster
def get_clusters(k, data):
   # Create a copy of the DataFrame
   data = data.copy()

   # Initialize the K-Means model
   model = KMeans(n_clusters=k, random_state=0)

   # Fit the model
   model.fit(data)

   # Predict clusters
   predictions = model.predict(data)

   # Create return DataFrame with predicted clusters
   data["class"] = model.labels_

   return data